In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

/home/romulo/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [69]:
df = pd.read_csv('../src/data/w241_Enriched_Tracker_Based_on_Emails.csv')

def clean_rental_data(df):
    cln_df = (df[['numberOfResponsesToThisTreatment', 'Nonwhite', 'Child', 
                  'Posting Date', 'ignore', 'neighborhood']]
                  .query('ignore == False')
                  .rename(columns={'neighborhood': 'Neighborhood',
                                   'Posting Date': 'PostingDate'})
                  .assign(PostingDate=lambda x: pd.to_datetime(x['PostingDate']),
                          SentDate=pd.to_datetime('2017-11-21', format='%Y-%m-%d'),
                          Response=lambda x: (x['numberOfResponsesToThisTreatment'] > 0).astype(int))              
                  .assign(NumDaysUp=lambda x: (x['SentDate'] - x['PostingDate']).dt.days)
                  .drop(labels=['numberOfResponsesToThisTreatment', 'PostingDate',
                                'SentDate', 'ignore'], axis=1))
    return cln_df

cln_df = clean_rental_data(df)

In [70]:
df

,Unnamed: 0,Listed By,Listed By Cleaned,neighborhood,Posting Date,Scrape Date,Source File Name,Title,url,Nonwhite,Child,Email Address,Email Status,inbox,treatmentEmail,ignore,multipleTreatmentResponses,multipleTreatmentApplied,numberOfResponsesToThisTreatment,numberOfEmailsSentForThisTreatment
0,0,106fulton_elena,106fultonelena,FinancialDistrict,11/11/2017,11/19/2017,craigslist_newyork_6383039355_20171119.json,Large Bright New Studio flex 1 Bed/flex 2 Bed,https://newyork.craigslist.org/mnh/abo/d/large...,0,0,3tth9-6383039355@hous.craigslist.org,SENT,bernard.c.gibson@gmail.com,3tth9-6383039355@hous.craigslist.org,False,False,False,1.0,1.0
1,1,1071 First Ave LLC - Julia,1071firstavellcjulia,MidtownEast,10/25/2017,11/19/2017,craigslist_newyork_6360717090_20171119.json,1071 1st Ave.(Bet. 58th and 59th) 2 beds and 2...,https://newyork.craigslist.org/mnh/abo/d/1071-...,1,1,3qbbf-6360717090@hous.craigslist.org,NaN,alej.gonzalez.47@gmail.com,3qbbf-6360717090@hous.craigslist.org,False,True,True,1.0,1.0
2,2,15 Cliff Management- Greystar,15cliffmanagementgreystar,FinancialDistrict,11/18/2017,11/19/2017,craigslist_newyork_6392330376_20171119.json,"Refrigerator W/ Wine Rack, Contemporary Vanity...",https://newyork.craigslist.org/mnh/abo/d/refri...,0,1,vpfvr-6392330376@hous.craigslist.org,NaN,bernie.j.smith.37@gmail.com,vpfvr-6392330376@hous.craigslist.org,False,False,False,0.0,1.0
3,3,(1800 sq.ft) 2BR/2BTH! WASHER/DRYER IN UNIT! B...,1800sqft2br2bthwasherdryerinunitbalcony,SoHo,11/6/2017,11/19/2017,craigslist_newyork_6376838702_20171119.json,HUGE 2BR/2BTH! WASHER/DRYER IN UNIT! DOORMAN! ...,https://newyork.craigslist.org/mnh/abo/d/huge-...,0,1,rwvzb-6376838702@hous.craigslist.org,NaN,bernie.j.smith.37@gmail.com,rwvzb-6376838702@hous.craigslist.org,False,False,False,0.0,1.0
4,4,................2BDR 2 BATH,2bdr2bath,MidtownEast,11/4/2017,11/19/2017,craigslist_newyork_6373703113_20171119.json,"SPACIOUS 2BDR 2 BATH, GREAT SUNLIGHT, DRMAN, P...",https://newyork.craigslist.org/mnh/abo/d/spaci...,0,0,n4zc5-6373703113@hous.craigslist.org,SENT,bernard.c.gibson@gmail.com,n4zc5-6373703113@hous.craigslist.org,False,False,False,1.0,1.0
5,5,2 bedroom apartment on 108th street no fee,2bedroomapartmenton108thstreetnofee,UpperWestSide,11/9/2017,11/19/2017,craigslist_newyork_6380500730_20171119.json,"w108@Amsterdam, 2 bed, bright, clean, wonderfu...",https://newyork.craigslist.org/mnh/abo/d/w108a...,0,1,dvhzp-6380500730@hous.craigslist.org,NaN,bernie.j.smith.37@gmail.com,dvhzp-6380500730@hous.craigslist.org,False,False,False,0.0,1.0
6,6,2 BEDROOM ON UPPER EAST SIDE,2bedroomonuppereastside,UpperEastSide,11/11/2017,11/19/2017,craigslist_newyork_6382892867_20171119.json,NO FEE! AMAZING 2BR! NEWLY RENOVATED! DOORMAN!...,https://newyork.craigslist.org/mnh/abo/d/no-fe...,0,1,8dxzh-6382892867@hous.craigslist.org,NaN,bernie.j.smith.37@gmail.com,8dxzh-6382892867@hous.craigslist.org,False,False,False,0.0,1.0
7,7,2 Bedrooms Apartment,2bedroomsapartment,Flatiron,10/25/2017,11/19/2017,craigslist_newyork_6360764974_20171119.json,2 Bedroom apt DM Building Most Prime location,https://newyork.craigslist.org/mnh/abo/d/2-bed...,1,1,gnfrh-6360764974@hous.craigslist.org,NaN,alej.gonzalez.47@gmail.com,gnfrh-6360764974@hous.craigslist.org,True,False,False,1.0,1.0
8,8,2 BR /2 bath IN HEART OF MIDTOWN HUGE LIVING ROOM,2br2bathinheartofmidtownhugelivingroom,UpperEastSide,11/11/2017,11/19/2017,craigslist_newyork_6382320300_20171119.json,2 BR /2 bath IN HEART OF MIDTOWN ~ HUGE LIVING...,https://newyork.craigslist.org/mnh/abo/d/2-br-...,1,1,4bm6b-6382320300@hous.craigslist.org,NaN,alej.gonzalez.47@gmail.com,4bm6b-6382320300@hous.craigslist.org,False,False,False,0.0,1.0
9,9,2BR APARTMENT! SS APPLIANCES! LUNDRY! DOORMAN!,2brapartmentssapplianceslundrydoorman,Gramercy,11/11/2017,11/19/2017,craigslist_newyork_6383528147_20171119.json,NO FEE! 2BR APARTMENT! SS APPLIANCES! LUNDRY! ...,https://newyork.craigslist.org/mnh/abo/d/no-fe...,1,0,p6bpv-6383528147@hous.craigslist.org,NaN,ramirezjalej@

In [67]:
dropped = df[['numberOfResponsesToThisTreatment', 'Nonwhite', 'Child', 
                  'Posting Date', 'ignore', 'neighborhood','multipleTreatmentApplied']].query('ignore == True').rename(columns={'neighborhood': 'Neighborhood',
                                   'Posting Date': 'PostingDate'}).assign(PostingDate=lambda x: pd.to_datetime(x['PostingDate']),
                          SentDate=pd.to_datetime('2017-11-21', format='%Y-%m-%d'),
                          Response=lambda x: (x['numberOfResponsesToThisTreatment'] > 0).astype(int)).assign(NumDaysUp=lambda x: (x['SentDate'] - x['PostingDate']).dt.days).drop(labels=['numberOfResponsesToThisTreatment', 'PostingDate',
                                'SentDate', 'ignore'], axis=1)

dropped['Subject'] = 1
droppedAgg = dropped.groupby(['Nonwhite', 'Child','Neighborhood']).agg({'Subject':'sum','Response':'sum'}).reset_index()
droppedAgg['responseRate'] = (droppedAgg['Response']/droppedAgg['Subject'])
pd.pivot_table(droppedAgg,values='Subject',index=['Nonwhite'],columns=['Child'],aggfunc=np.sum)

Child,0,1
Nonwhite,,
0,1,5
1,11,10


In [68]:
dropped

,Nonwhite,Child,Neighborhood,multipleTreatmentApplied,Response,NumDaysUp,Subject
7,1,1,Flatiron,False,1,27,1
13,1,1,EastVillage,False,1,32,1
34,1,1,Inwood/WashHts,True,1,34,1
37,1,0,Inwood/WashHts,False,1,8,1
68,0,1,LowerEastSide,False,1,19,1
70,1,1,UpperEastSide,False,1,8,1
103,1,0,BatteryPark,False,1,19,1
132,1,0,Harlem/Morningside,False,1,8,1
143,1,0,Inwood/WashHts,False,1,8,1
145,0,1,Inwood/WashHts,False,1,34,1


In [4]:
cln_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 0 to 316
Data columns (total 5 columns):
Nonwhite        290 non-null int64
Child           290 non-null int64
Neighborhood    290 non-null object
Response        290 non-null int64
NumDaysUp       290 non-null int64
dtypes: int64(4), object(1)
memory usage: 23.6+ KB


In [ ]:
aggfuncs = {
    'Response': np.sum,
    'Response': np.
}

In [79]:
cln_df.groupby(['Nonwhite', 'Child']).count()['Response'].reset_index()

,Nonwhite,Child,Neighborhood
0,0,0,82
1,0,1,74
2,1,0,63
3,1,1,71


In [83]:
cln_df.groupby(['Nonwhite', 'Child']).sum()['Response'].reset_index()

,Nonwhite,Child,Response
0,0,0,66
1,0,1,43
2,1,0,38
3,1,1,41


In [85]:
print(66 / 82)
print(43 / 74)
print(38 / 63)
print(41 / 71)

0.8048780487804879
0.581081081081081
0.6031746031746031
0.5774647887323944


In [22]:
cln_df.head()
cln_df['Subject'] = 1


In [23]:
cln_df.to_csv('cleaned_final_data.csv', index=False)

In [29]:

cln_df_plt = cln_df.groupby(['Nonwhite', 'Child','Neighborhood']).agg({'Subject':'sum','Response':'sum'}).reset_index()
cln_df_plt['responseRate'] = (cln_df_plt['Response']/ cln_df_plt['Subject'])

In [34]:
cln_df_plt[(cln_df_plt['Nonwhite'] == True) & (cln_df_plt['Child'] == True)].reset_index()

,index,Nonwhite,Child,Neighborhood,Subject,Response,responseRate
0,57,1,1,Chelsea,2,1,0.500000
1,58,1,1,Chinatown/LitItaly,1,0,0.000000
2,59,1,1,Downtown,1,1,1.000000
3,60,1,1,EastHarlem,4,1,0.250000
4,61,1,1,EastVillage,3,1,0.333333
5,62,1,1,FinancialDistrict,2,1,0.500000
6,63,1,1,Gramercy,4,2,0.500000
7,64,1,1,GreenwichVillage,1,0,0.000000
8,65,1,1,Harlem/Morningside,4,4,1.000000
9,66,1,1,Inwood/WashHts,7,3,0.428571


In [42]:
pd.pivot_table(cln_df_plt,values='responseRate',index=['Neighborhood'],columns=['Nonwhite','Child'])

Nonwhite                   0                   1          
Child                      0         1         0         1
Neighborhood                                              
BatteryPark         1.000000       NaN       NaN       NaN
Chelsea             0.666667  1.000000  0.666667  0.500000
Chinatown/LitItaly       NaN       NaN       NaN  0.000000
Downtown                 NaN       NaN       NaN  1.000000
EastHarlem          0.800000  1.000000  1.000000  0.250000
EastVillage         0.750000  0.200000  0.400000  0.333333
FinancialDistrict   1.000000  0.500000  0.500000  0.500000
Gramercy            1.000000  1.000000  0.250000  0.500000
GreenwichVillage    1.000000  1.000000  1.000000  0.000000
Harlem/Morningside  0.800000  0.400000  0.500000  1.000000
Inwood/WashHts      0.750000  0.666667  0.800000  0.428571
LowerEastSide       0.666667  0.500000  0.333333  0.500000
Midtown             1.000000  0.500000  0.500000  1.000000
MidtownEast         0.714286  0.857143  0.500000  0.500000
MidtownWest         1.000000  0.800000  0.833333  0.666667
MurrayHill          0.875000  0.500000  0.666667  0.571429
Nolita/Bowery       1.000000       NaN  1.000000       NaN
SoHo                     NaN  0.000000  0.500000  1.000000
TriBeCa             1.000000  1.000000       NaN  1.000000
UnionSquare         1.000000  0.000000       NaN  1.000000
UpperEastSide       0.555556  0.300000  0.700000  0.666667
UpperWestSide       0.857143  0.714286  0.333333  0.571429
WestVillage         1.000000  0.000000  0.000000  1.000000

In [25]:
%%html
<iframe
  width="800"
  height="400"
  seamless
  frameBorder="0"
  scrolling="no"
  src="http://localhost:8089/superset/explore/table/8/?preselect_filters=%7B%7D&form_data=%7B%22datasource%22%3A%228__table%22%2C%22viz_type%22%3A%22dist_bar%22%2C%22slice_id%22%3A42%2C%22granularity_sqla%22%3Anull%2C%22time_grain_sqla%22%3Anull%2C%22since%22%3A%227+days+ago%22%2C%22until%22%3A%22now%22%2C%22metrics%22%3A%5B%22sum__Response%22%2C%22sum__Subject%22%5D%2C%22groupby%22%3A%5B%22Neighborhood%22%5D%2C%22columns%22%3A%5B%5D%2C%22row_limit%22%3Anull%2C%22color_scheme%22%3A%22bnbColors%22%2C%22show_legend%22%3Atrue%2C%22show_bar_value%22%3Afalse%2C%22bar_stacked%22%3Afalse%2C%22order_bars%22%3Afalse%2C%22y_axis_format%22%3A%22.3s%22%2C%22bottom_margin%22%3A%22auto%22%2C%22x_axis_label%22%3A%22%22%2C%22y_axis_label%22%3A%22%22%2C%22reduce_x_ticks%22%3Afalse%2C%22contribution%22%3Afalse%2C%22show_controls%22%3Afalse%2C%22where%22%3A%22%22%2C%22having%22%3A%22%22%2C%22filters%22%3A%5B%5D%7D&standalone=true&height=400"
>
</iframe>